# 🚀 AlgVex 加密货币量化交易教程

## Qlib + Hummingbot 融合的加密货币现货量化交易平台

**版本**: v10.0.4 (对应 CORE_PLAN v10.0.4)  
**更新日期**: 2026-01-03  
**适用对象**: 有一定 Python 基础的量化交易学习者

---

## 🎯 本教程特点

| 特点 | 说明 |
|------|------|
| **零修改策略** | Qlib 和 Hummingbot 均无源码修改 |
| **回测=实盘** | 训练/回测/实盘使用完全相同的特征链路 |
| **统一数据格式** | Parquet 离线数据 + Hummingbot 实时数据 |
| **V2 架构** | 使用 Hummingbot Strategy V2 框架 |

---

## 📚 教程结构

```
Part 0: 环境搭建与验证 (30分钟)
├── 依赖安装
├── 系统验证
└── 目录结构

Part 1: 数据准备 (30分钟)
├── 获取历史数据
├── Parquet 格式说明
└── 数据质量检查

Part 2: 统一特征计算 (45分钟)
├── 59个因子详解
├── 特征归一化
└── 防泄漏机制

Part 3: 模型训练 (45分钟)
├── LightGBM 训练
├── 模型保存
└── 验证 IC

Part 4: 离线回测 (45分钟)
├── 回测原理
├── 仿真交易
└── 指标分析

Part 5: Hummingbot 实盘 (60分钟)
├── V2 架构介绍
├── Controller 配置
└── Paper Trading
```

---

## ⚠️ 核心原则 (v10.0.0)

```
┌─────────────────────────────────────────────────────────────┐
│                      核心原则                                │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  1. 零修改策略                                               │
│     - Qlib: 运行时配置覆盖，无源码修改                      │
│     - Hummingbot: 零修改，使用 Strategy V2 框架             │
│                                                             │
│  2. 唯一数据格式                                             │
│     - 离线数据: Parquet 格式 (非 Qlib .bin)                 │
│     - 实盘数据: Hummingbot candles                          │
│                                                             │
│  3. 回测=实盘                                                │
│     - 训练/回测/实盘使用完全相同的特征计算链路              │
│     - unified_features → normalizer → Booster.predict       │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

---

# 📘 Part 0: 环境搭建与验证

> **预计时间**: 30分钟  
> **目标**: 确保所有依赖正确安装，系统可正常运行

## 0.1 安装依赖

运行以下命令安装必要的 Python 包：

In [ ]:
# ============================================================
#            📦 安装依赖
# ============================================================

# 核心依赖
!pip install -q numpy pandas lightgbm pyarrow aiohttp

# 可视化
!pip install -q matplotlib plotly

# Qlib (可选，用于高级研究)
# !pip install pyqlib

print("✓ 依赖安装完成")

## 0.2 验证环境

In [ ]:
# ============================================================
#            🔍 环境验证
# ============================================================

import sys
print("="*60)
print("🎉 AlgVex v10.0.4 环境验证")
print("="*60)

print(f"\n✅ Python 版本: {sys.version}")

# 检查核心包
packages = [
    ("numpy", "np"),
    ("pandas", "pd"),
    ("lightgbm", "lgb"),
    ("pyarrow", "pa"),
]

print("\n📦 核心包状态:")
for pkg_name, alias in packages:
    try:
        pkg = __import__(pkg_name)
        version = getattr(pkg, "__version__", "unknown")
        print(f"   ✅ {pkg_name}: {version}")
    except ImportError:
        print(f"   ❌ {pkg_name}: 未安装")

# 检查 Qlib (可选)
try:
    import qlib
    print(f"   ✅ qlib: {qlib.__version__} (可选)")
except ImportError:
    print(f"   ⚠️ qlib: 未安装 (可选，主流程不依赖)")

print("\n" + "="*60)

## 0.3 目录结构

AlgVex v10.0.4 使用以下目录结构：

```
~/.algvex/
├── data/
│   └── 1h/                      # 按频率组织
│       ├── btcusdt.parquet      # 每个交易对一个文件
│       ├── ethusdt.parquet
│       └── metadata.json        # 数据元信息
└── models/
    └── qlib_alpha/              # 按策略名组织
        ├── lgb_model.txt        # LightGBM 模型
        ├── normalizer.pkl       # 归一化参数
        ├── feature_columns.pkl  # 特征列顺序
        └── metadata.json        # 训练参数
```

In [ ]:
# ============================================================
#            📁 创建目录结构
# ============================================================

from pathlib import Path

# 创建目录
DATA_DIR = Path.home() / ".algvex" / "data" / "1h"
MODEL_DIR = Path.home() / ".algvex" / "models" / "qlib_alpha"

DATA_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)

print(f"✅ 数据目录: {DATA_DIR}")
print(f"✅ 模型目录: {MODEL_DIR}")

## 0.4 运行集成验证脚本

In [ ]:
# ============================================================
#            ✅ 运行集成验证
# ============================================================

# 切换到项目目录
import os
os.chdir('/home/user/AlgVex_ATS')

# 运行验证脚本
!python scripts/verify_integration.py

---

# 📘 Part 1: 数据准备

> **预计时间**: 30分钟  
> **目标**: 获取历史 K 线数据，保存为 Parquet 格式

## 1.1 数据来源

AlgVex v10.0.4 使用 Binance API 获取历史数据：

| 数据类型 | 来源 | 频率 | 格式 |
|----------|------|------|------|
| 历史 K 线 | Binance API | 1h/4h/1d | Parquet |
| 实时 K 线 | Hummingbot | 1h | DataFrame |

### 为什么使用 Parquet 而非 Qlib .bin？

| 对比维度 | Qlib .bin | Parquet (本方案) |
|----------|-----------|------------------|
| **可读性** | 二进制，需专用工具 | 可直接用 pandas 读取 |
| **调试性** | 难以验证对齐 | 可直接查看 DataFrame |
| **依赖** | 必须用 Qlib Provider | 独立于 Qlib |

## 1.2 获取历史数据

使用 `prepare_crypto_data.py` 脚本获取数据：

```bash
python scripts/prepare_crypto_data.py \
    --trading-pairs BTC-USDT ETH-USDT \
    --interval 1h \
    --start-date 2023-01-01 \
    --end-date 2024-12-31
```

由于网络限制，这里使用模拟数据进行演示：

In [ ]:
# ============================================================
#            📥 生成模拟数据 (演示用)
# ============================================================

import json
import numpy as np
import pandas as pd
from pathlib import Path

# 配置
DATA_DIR = Path.home() / ".algvex" / "data" / "1h"
DATA_DIR.mkdir(parents=True, exist_ok=True)

# 生成 2 年的 1h 数据
np.random.seed(42)
n = 17520  # 2 years * 365 days * 24 hours

def generate_ohlcv(base_price, n, volatility=0.02, drift=0.0001):
    """生成 OHLCV 数据"""
    returns = np.random.normal(drift, volatility, n)
    close = base_price * np.exp(np.cumsum(returns))
    
    df = pd.DataFrame({
        "open": close * (1 + np.random.uniform(-0.005, 0.005, n)),
        "high": close * (1 + np.random.uniform(0, 0.02, n)),
        "low": close * (1 - np.random.uniform(0, 0.02, n)),
        "close": close,
        "volume": np.random.uniform(100, 1000, n),
        "quote_volume": np.random.uniform(4000000, 40000000, n),
    }, index=pd.date_range("2023-01-01", periods=n, freq="1h", tz="UTC"))
    
    return df

# 生成 BTC 和 ETH 数据
btc_df = generate_ohlcv(40000, n, volatility=0.02)
eth_df = generate_ohlcv(2500, n, volatility=0.025)

# 保存
btc_df.to_parquet(DATA_DIR / "btcusdt.parquet", engine="pyarrow")
eth_df.to_parquet(DATA_DIR / "ethusdt.parquet", engine="pyarrow")

# 保存元数据
metadata = {
    "freq": "1h",
    "timezone": "UTC",
    "instruments": [
        {"name": "btcusdt", "rows": len(btc_df)},
        {"name": "ethusdt", "rows": len(eth_df)},
    ],
    "columns": ["open", "high", "low", "close", "volume", "quote_volume"],
}
with open(DATA_DIR / "metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)

print(f"✅ btcusdt.parquet: {len(btc_df)} 行")
print(f"✅ ethusdt.parquet: {len(eth_df)} 行")
print(f"✅ 数据保存到: {DATA_DIR}")

## 1.3 数据质量检查

In [ ]:
# ============================================================
#            🔍 数据质量检查
# ============================================================

import pandas as pd
from pathlib import Path

DATA_DIR = Path.home() / ".algvex" / "data" / "1h"

for file_path in DATA_DIR.glob("*.parquet"):
    df = pd.read_parquet(file_path)
    
    print(f"\n{'='*60}")
    print(f"📊 {file_path.name}")
    print(f"{'='*60}")
    
    # 基本信息
    print(f"\n📈 基本信息:")
    print(f"   行数: {len(df):,}")
    print(f"   时间范围: {df.index.min()} to {df.index.max()}")
    print(f"   时区: {df.index.tz}")
    
    # 数据质量
    print(f"\n🔍 数据质量:")
    print(f"   缺失值: {df.isna().sum().sum()}")
    print(f"   重复行: {df.index.duplicated().sum()}")
    print(f"   时间有序: {'✅' if df.index.is_monotonic_increasing else '❌'}")
    
    # 价格统计
    print(f"\n💰 价格统计:")
    print(f"   开盘: ${df['open'].iloc[0]:,.2f} → ${df['open'].iloc[-1]:,.2f}")
    print(f"   最高: ${df['high'].max():,.2f}")
    print(f"   最低: ${df['low'].min():,.2f}")

## 1.4 数据可视化

In [ ]:
# ============================================================
#            📈 价格走势可视化
# ============================================================

import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

DATA_DIR = Path.home() / ".algvex" / "data" / "1h"

# 读取数据
btc_df = pd.read_parquet(DATA_DIR / "btcusdt.parquet")

# 创建图表
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# 价格图
axes[0].plot(btc_df.index, btc_df['close'], label='Close', linewidth=0.8)
axes[0].set_ylabel('Price (USD)')
axes[0].set_title('BTC-USDT Price')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 成交量图
axes[1].bar(btc_df.index, btc_df['volume'], width=0.02, alpha=0.7)
axes[1].set_ylabel('Volume')
axes[1].set_xlabel('Date')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

# 📘 Part 2: 统一特征计算

> **预计时间**: 45分钟  
> **目标**: 理解并计算 59 个统一特征

## 2.1 特征概述

AlgVex v10.0.4 使用 59 个统一特征，分为 11 大类：

| 类别 | 因子名 | 数量 | 说明 |
|------|--------|------|------|
| **KBAR** | KMID, KLEN, KMID2, KUP, KUP2, KLOW, KLOW2, KSFT, KSFT2 | 9 | K 线形态 |
| **ROC** | ROC5, ROC10, ROC20, ROC30, ROC60 | 5 | 变化率 |
| **MA** | MA5, MA10, MA20, MA30, MA60 | 5 | 均线偏离 |
| **STD** | STD5, STD10, STD20, STD30, STD60 | 5 | 波动率 |
| **MAX** | MAX5, MAX10, MAX20, MAX30, MAX60 | 5 | 最高价偏离 |
| **MIN** | MIN5, MIN10, MIN20, MIN30, MIN60 | 5 | 最低价偏离 |
| **QTLU** | QTLU5, QTLU10, QTLU20, QTLU30, QTLU60 | 5 | 上分位偏离 |
| **QTLD** | QTLD5, QTLD10, QTLD20, QTLD30, QTLD60 | 5 | 下分位偏离 |
| **RSV** | RSV5, RSV10, RSV20, RSV30, RSV60 | 5 | 随机指标 |
| **CORR** | CORR5, CORR10, CORR20, CORR30, CORR60 | 5 | 价量相关 |
| **CORD** | CORD5, CORD10, CORD20, CORD30, CORD60 | 5 | 收益量相关 |
| **总计** | | **59** | |

In [ ]:
# ============================================================
#            📐 导入统一特征模块
# ============================================================

import sys
sys.path.insert(0, '/home/user/AlgVex_ATS')

from scripts.unified_features import (
    compute_unified_features,
    compute_label,
    FeatureNormalizer,
    FEATURE_COLUMNS,
)

print(f"✅ 特征列数: {len(FEATURE_COLUMNS)}")
print(f"\n📋 特征列表:")
for i, col in enumerate(FEATURE_COLUMNS):
    print(f"   {i+1:2d}. {col}")

## 2.2 计算特征

In [ ]:
# ============================================================
#            📊 计算统一特征
# ============================================================

import pandas as pd
from pathlib import Path

# 读取数据
DATA_DIR = Path.home() / ".algvex" / "data" / "1h"
btc_df = pd.read_parquet(DATA_DIR / "btcusdt.parquet")

print(f"📥 加载数据: {len(btc_df)} 行")

# 计算特征
features = compute_unified_features(btc_df)
labels = compute_label(btc_df)

print(f"\n📐 特征矩阵:")
print(f"   形状: {features.shape}")
print(f"   NaN 行数: {features.isna().any(axis=1).sum()} (前 60 行因滚动窗口)")

# 剔除 NaN
valid_mask = ~features.isna().any(axis=1)
features_valid = features[valid_mask]
labels_valid = labels[valid_mask]

print(f"   有效行数: {len(features_valid)}")

# 显示特征统计
print(f"\n📊 特征统计 (前 5 个):")
print(features_valid[FEATURE_COLUMNS[:5]].describe())

## 2.3 特征归一化

### FeatureNormalizer 类

- `fit_transform()`: 训练时使用，计算并保存均值/标准差
- `transform(strict=True)`: 预测时使用，应用保存的参数

⚠️ **防泄漏**: 只能用训练集调用 `fit_transform()`！

In [ ]:
# ============================================================
#            🔧 特征归一化演示
# ============================================================

# 创建归一化器
normalizer = FeatureNormalizer()

# 模拟训练集/测试集分割
train_size = int(len(features_valid) * 0.8)
train_features = features_valid.iloc[:train_size]
test_features = features_valid.iloc[train_size:]

print(f"📊 训练集: {len(train_features)} 行")
print(f"📊 测试集: {len(test_features)} 行")

# 训练时: fit_transform
train_norm = normalizer.fit_transform(train_features)
print(f"\n✅ 训练集归一化完成")
print(f"   均值范围: [{normalizer.mean.min():.4f}, {normalizer.mean.max():.4f}]")
print(f"   标准差范围: [{normalizer.std.min():.4f}, {normalizer.std.max():.4f}]")

# 测试时: transform (strict=True)
test_norm = normalizer.transform(test_features, strict=True)
print(f"\n✅ 测试集归一化完成 (strict=True)")

# 验证归一化后的统计
print(f"\n📊 归一化后的统计:")
print(f"   训练集均值: {train_norm.mean().mean():.6f} (应接近 0)")
print(f"   训练集标准差: {train_norm.std().mean():.6f} (应接近 1)")

## 2.4 特征可视化

In [ ]:
# ============================================================
#            📈 特征分布可视化
# ============================================================

import matplotlib.pyplot as plt

# 选择几个代表性特征
sample_features = ["KMID", "ROC20", "MA20", "STD20", "RSV20"]

fig, axes = plt.subplots(1, 5, figsize=(15, 3))

for i, feat in enumerate(sample_features):
    axes[i].hist(train_norm[feat].dropna(), bins=50, alpha=0.7)
    axes[i].set_title(feat)
    axes[i].set_xlabel('Normalized Value')

plt.suptitle('Feature Distributions (Normalized)', y=1.02)
plt.tight_layout()
plt.show()

---

# 📘 Part 3: 模型训练

> **预计时间**: 45分钟  
> **目标**: 使用 LightGBM 训练预测模型

## 3.1 训练流程

```
┌─────────────────────────────────────────────────────────────┐
│              模型训练流程 (v10.0.0)                          │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  1. 加载 Parquet 数据                                       │
│  2. 计算统一特征 (compute_unified_features)                 │
│  3. 计算标签 (t+1 收益率)                                   │
│  4. 时间切分 (训练集 / 验证集)                              │
│  5. 归一化 (仅在训练集 fit)                                 │
│  6. LightGBM 训练                                           │
│  7. 保存模型 4 件套                                         │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
# ============================================================
#            🚀 运行模型训练
# ============================================================

import os
os.chdir('/home/user/AlgVex_ATS')

!python scripts/train_model.py \
    --data-dir ~/.algvex/data \
    --output-dir ~/.algvex/models/qlib_alpha \
    --instruments btcusdt ethusdt \
    --train-start 2023-01-01 \
    --train-end 2024-06-15 \
    --valid-start 2024-07-01 \
    --valid-end 2024-12-31 \
    --freq 1h

## 3.2 验证模型文件

In [ ]:
# ============================================================
#            📦 验证模型文件
# ============================================================

import json
import pickle
from pathlib import Path
import lightgbm as lgb

MODEL_DIR = Path.home() / ".algvex" / "models" / "qlib_alpha"

print("📦 模型文件:")
for f in MODEL_DIR.iterdir():
    print(f"   {f.name}: {f.stat().st_size:,} bytes")

# 加载模型
model = lgb.Booster(model_file=str(MODEL_DIR / "lgb_model.txt"))
print(f"\n✅ LightGBM 模型: {model.num_trees()} 棵树")

# 加载元数据
with open(MODEL_DIR / "metadata.json") as f:
    metadata = json.load(f)

print(f"\n📋 训练元数据:")
for key, value in metadata.items():
    print(f"   {key}: {value}")

---

# 📘 Part 4: 离线回测

> **预计时间**: 45分钟  
> **目标**: 使用历史数据验证策略表现

## 4.1 回测原则

```
┌─────────────────────────────────────────────────────────────┐
│              回测 = 实盘 (v10.0.0)                           │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  唯一链路 (离线回测与实盘完全一致):                         │
│  ───────────────────────────────                            │
│  1. OHLCV (Parquet) → compute_unified_features()            │
│  2. features[feature_columns] (严格对齐，缺列 FAIL)         │
│  3. normalizer.transform(strict=True)                       │
│  4. booster.predict(X) → pred                               │
│  5. signal = +1/-1/0 (阈值与实盘一致)                       │
│  6. 仓位仿真 (手续费/滑点/止损止盈/时间限制/冷却)           │
│                                                             │
│  ⚠️ 必须使用"已收盘bar" (iloc[-2])，与实盘一致             │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
# ============================================================
#            📈 运行离线回测
# ============================================================

import os
os.chdir('/home/user/AlgVex_ATS')

!python scripts/backtest_offline.py \
    --data-dir ~/.algvex/data \
    --model-dir ~/.algvex/models/qlib_alpha \
    --instruments btcusdt ethusdt \
    --test-start 2024-07-01 \
    --test-end 2024-12-31 \
    --freq 1h \
    --signal-threshold 0.001 \
    --stop-loss 0.02 \
    --take-profit 0.03

## 4.2 回测指标说明

| 指标 | 含义 | 通过标准 |
|------|------|----------|
| **胜率** | 盈利交易占比 | > 40% |
| **夏普比率** | 风险调整后收益 | > 0.5 |
| **最大回撤** | 最大亏损幅度 | < 30% |
| **平均收益** | 每笔交易平均收益 | > 0 |

> **注意**: 使用模拟随机数据时，指标可能不达标。使用真实 Binance 数据时，模型应能学到有意义的价格模式。

---

# 📘 Part 5: Hummingbot 实盘交易

> **预计时间**: 60分钟  
> **目标**: 配置并运行 Hummingbot Strategy V2

## 5.1 V2 架构介绍

```
┌─────────────────────────────────────────────────────────────┐
│              Hummingbot Strategy V2 架构                     │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  StrategyV2Base                                             │
│      │                                                      │
│      ├── MarketDataProvider  ← 统一数据获取接口             │
│      │   └── get_candles_df()                               │
│      │                                                      │
│      ├── Controller          ← Qlib 模型预测                │
│      │   └── QlibAlphaController                            │
│      │                                                      │
│      └── Executors           ← 订单执行组件                 │
│          └── PositionExecutor (止损/止盈/时间限制)          │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

## 5.2 配置文件

### 控制器配置 (`conf/controllers/qlib_alpha.yml`)

In [ ]:
# ============================================================
#            📄 查看控制器配置
# ============================================================

!cat /home/user/AlgVex_ATS/conf/controllers/qlib_alpha.yml

### 策略配置 (`conf/scripts/qlib_alpha_v2.yml`)

In [ ]:
# ============================================================
#            📄 查看策略配置
# ============================================================

!cat /home/user/AlgVex_ATS/conf/scripts/qlib_alpha_v2.yml

## 5.3 启动步骤

```bash
# Step 1: 进入 Hummingbot 目录
cd hummingbot && ./start

# Step 2: 连接交易所 (Paper Trading)
>>> connect binance_paper_trade

# Step 3: 启动策略
>>> start --script qlib_alpha_strategy.py --conf conf/scripts/qlib_alpha_v2.yml

# Step 4: 查看状态
>>> status
```

## 5.4 Controller 核心代码解析

### 信号生成 (`_get_signal`)

```python
def _get_signal(self) -> int:
    # 1. 获取 K 线数据
    candles = self.processed_data.get("candles")
    
    # 2. 计算统一特征 (与训练完全一致)
    features = compute_unified_features(candles)
    
    # 3. 取倒数第二行 (已收盘 K 线)
    latest_features = features.iloc[-2:-1]
    
    # 4. 归一化 (strict=True)
    latest_features_norm = self.normalizer.transform(latest_features, strict=True)
    
    # 5. 预测
    prediction = self.model.predict(latest_features_norm.values)[0]
    
    # 6. 根据阈值生成信号
    if prediction > threshold:
        return 1  # 买入
    elif prediction < -threshold:
        return -1  # 卖出
    return 0  # 持有
```

---

# 📘 Part 6: 验收测试

> **预计时间**: 30分钟  
> **目标**: 验证系统完整性

In [ ]:
# ============================================================
#            ✅ 运行完整验收测试
# ============================================================

import os
os.chdir('/home/user/AlgVex_ATS')

!python scripts/verify_integration.py

## 6.1 验收清单

| 序号 | 验收项 | 验收方法 | 通过标准 |
|------|--------|----------|----------|
| 1 | 数据准备 | 运行 prepare_crypto_data.py | `*.parquet` 生成 |
| 2 | 模型训练 | 运行 train_model.py | 4 件套生成 |
| 3 | 离线回测 | 运行 backtest_offline.py | Sharpe > 0.5, MaxDD < 30% |
| 4 | Qlib 初始化 | Controller 启动 | region="us" |
| 5 | MarketDataProvider | Controller 运行 | ≥ 61 条数据 |
| 6 | 特征计算 | Controller 运行 | 59 特征非空 |
| 7 | Normalizer | Controller 运行 | strict=True 无异常 |
| 8 | 信号生成 | Controller 运行 | iloc[-2] ±1/0 |
| 9 | PositionExecutor | 策略运行 | Decimal 精度 |
| 10 | 三重屏障 | 触发条件 | Executor 自动关闭 |
| 11 | Paper Trading | 模拟交易 24h | 无异常 |

---

# 🎉 恭喜完成教程!

## 📋 你学到了什么

1. **环境搭建**: 安装依赖、创建目录结构
2. **数据准备**: 获取 Binance 数据、保存为 Parquet
3. **特征计算**: 59 个统一特征、归一化器使用
4. **模型训练**: LightGBM 训练、模型保存
5. **离线回测**: 与实盘同链路的回测
6. **实盘交易**: Hummingbot V2 架构配置

## 🚀 下一步

1. **获取真实数据**: 使用真实 Binance API 获取历史数据
2. **优化模型**: 调整 LightGBM 参数、尝试更多特征
3. **Paper Trading**: 在 Hummingbot 中运行 24h 模拟交易
4. **实盘交易**: 配置真实 API，小资金开始实盘

## 📚 参考资源

- **AlgVex CORE_PLAN**: `docs/CORE_PLAN.md`
- **Qlib 文档**: https://qlib.readthedocs.io/
- **Hummingbot V2**: https://docs.hummingbot.org/v2-strategies/

---

**版本**: v10.0.4  
**更新日期**: 2026-01-03